# Purpose

2022-11-09: Make some changes on filtering logic. We don't want to add clusters for all the subs with embeddings, we want to keep only highly active `seed` subreddits.

2022-07-05: Run **subreddit-level** clustering jobs in parallel with `hydra`.



# Imports & Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import logging
import os
from pathlib import Path

import numpy as np
import pandas as pd
import plotly
import seaborn as sns

import mlflow
import hydra

import subclu
from subclu.eda.aggregates import compare_raw_v_weighted_language
from subclu.utils import set_working_directory, get_project_subfolder
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.utils.mlflow_logger import MlflowLogger
from subclu.utils.hydra_config_loader import LoadHydraConfig
from subclu.utils.data_irl_style import (
    get_colormap, theme_dirl, 
    get_color_dict, base_colors_for_manual_labels,
    check_colors_used,
)
from subclu.data.data_loaders import LoadPosts, LoadSubreddits, create_sub_level_aggregates


# ===
# imports specific to this notebook
from collections import Counter
# import umap
# import openTSNE
# from openTSNE import TSNE

# import hdbscan

import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize  # if we normalize the data, euclidean distance is approx of cosine

from sklearn.cluster import KMeans, DBSCAN, OPTICS, AgglomerativeClustering

from subclu.models.clustering import ClusterEmbeddings
from subclu.models.clustering_utils import plot_elbow_and_get_k


print_lib_versions([hydra, np, pd, plotly, sklearn, sns, subclu])

python		v 3.7.10
===
hydra		v: 1.1.0
numpy		v: 1.19.5
pandas		v: 1.2.4
plotly		v: 4.14.3
sklearn		v: 0.24.1
seaborn		v: 0.11.1
subclu		v: 0.6.1


In [3]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Set sqlite database as MLflow URI

In [4]:
# use new class to initialize mlflow
mlf = MlflowLogger(tracking_uri='sqlite')
mlflow.get_tracking_uri()

'sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-100-2021-04-28-djb-eda-german-subs/mlruns.db'

## Get list of experiments with new function

In [5]:
mlf.list_experiment_meta(output_format='pandas').tail(9)

,experiment_id,name,artifact_location,lifecycle_stage
35,35,v0.6.0_mUSE_aggregates,gs://i18n-subreddit-clustering/mlflow/mlruns/35,active
36,36,v0.6.0_mUSE_clustering_test,gs://i18n-subreddit-clustering/mlflow/mlruns/36,active
37,37,v0.6.0_mUSE_clustering,gs://i18n-subreddit-clustering/mlflow/mlruns/37,active
38,38,v0.6.0_nearest_neighbors,gs://i18n-subreddit-clustering/mlflow/mlruns/38,active
39,39,v0.6.1_mUSE_aggregates_test,gs://i18n-subreddit-clustering/mlflow/mlruns/39,active
40,40,v0.6.1_mUSE_aggregates,gs://i18n-subreddit-clustering/mlflow/mlruns/40,active
41,41,v0.6.1_mUSE_clustering_test,gs://i18n-subreddit-clustering/mlflow/mlruns/41,active
42,42,v0.6.1_mUSE_clustering,gs://i18n-subreddit-clustering/mlflow/mlruns/42,active
43,43,v0.6.1_nearest_neighbors,gs://i18n-subreddit-clustering/mlflow/mlruns/43,active


## Get experiment IDs to use for clustering

Pick an embeddings run that covers the subreddits we want to cluster.

for v0.6.1:
`91ac7ca171024c779c0992f59470c81b`

In [6]:
%%time

df_mlf = mlf.search_all_runs(experiment_ids=[40])
df_mlf.shape

CPU times: user 59.1 ms, sys: 44.8 ms, total: 104 ms
Wall time: 1.99 s


(4, 43)

In [7]:
df_mlf.iloc[:5, :13]

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.cpu_count,metrics.memory_used_percent,metrics.memory_used,metrics.memory_free,metrics.df_v_subs-rows,metrics.df_v_post_comments-rows,metrics.df_v_subs-cols
0,2695f87ed3a14caa8e87ba572ad49782,40,RUNNING,gs://i18n-subreddit-clustering/mlflow/mlruns/40/2695f87ed3a14caa8e87ba572ad49782/artifacts,2022-11-08 07:07:09.216000+00:00,NaT,96.0,0.869350,1256177.0,150879.0,781653.0,53597817.0,514.0
1,8519652dc1aa46c0b49911bb04cf7ffe,40,RUNNING,gs://i18n-subreddit-clustering/mlflow/mlruns/40/8519652dc1aa46c0b49911bb04cf7ffe/artifacts,2022-11-08 03:51:17.460000+00:00,NaT,96.0,0.497697,719153.0,759036.0,781653.0,53597817.0,514.0
2,91ac7ca171024c779c0992f59470c81b,40,FINISHED,gs://i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts,2022-11-07 21:38:57.662000+00:00,2022-11-08 08:52:44.944000+00:00,96.0,0.552656,798566.0,1283359.0,781653.0,53597817.0,514.0
3,f7f796e6dc054b3f89027a5027ac227a,40,RUNNING,gs://i18n-subreddit-clustering/mlflow/mlruns/40/f7f796e6dc054b3f89027a5027ac227a/artifacts,2022-11-07 19:00:30.584000+00:00,NaT,96.0,0.001309,1892.0,1441183.0,781653.0,NaN,514.0


In [8]:
# mask_finished = df_mlf['status'] == 'FINISHED'
# mask_df_similarity_complete = ~df_mlf['metrics.df_sub_level_agg_a_post_only_similarity-rows'].isnull()

# df_mlf_clustering_candidates = df_mlf[mask_finished & mask_df_similarity_complete]
# df_mlf_clustering_candidates.shape

In [9]:
# cols_with_multiple_vals = df_mlf_clustering_candidates.columns[df_mlf_clustering_candidates.nunique(dropna=False) > 1]

# df_mlf_clustering_candidates[cols_with_multiple_vals]

# Inspect config for clustering job

This config should include:
- data to load for clustering
- parameters for clustering algo
- hydra overrides to run jobs in parallel

In [10]:
test_experiment = 'v0.6.1_mUSE_clustering_test'

cfg_cluster_test = LoadHydraConfig(
    config_name='clustering_v0.6.1_subreddit_base.yaml',
    config_path="../config",
    overrides=[
        f"mlflow_experiment_name={test_experiment}"
#         f"data_text_and_metadata=top_subreddits_2021_07_16",
#         f"data_embeddings_to_cluster=top_subs-2021_07_16-use_multi_lower_case_false_00",
    ],
)

print([k for k in cfg_cluster_test.config_dict.keys()])

['data_text_and_metadata', 'data_embeddings_to_cluster', 'clustering_algo', 'embeddings_to_cluster', 'n_sample_embedding_rows', 'n_max_clusters_to_check_for_optimal_k', 'filter_embeddings', 'mlflow_tracking_uri', 'mlflow_experiment_name', 'pipeline_config']


In [11]:
cfg_cluster_test.config_dict['n_max_clusters_to_check_for_optimal_k']

8000

In [15]:
# data with text/meta
cfg_cluster_test.config_dict['data_text_and_metadata']

{'dataset_name': 'v0.6.1 inputs. ~110k seed subreddits, ~340k with 3+ posts, ~700k total subreddits',
 'bucket_name': 'i18n-subreddit-clustering',
 'folder_subreddits_text_and_meta': 'i18n_topic_model_batch/runs/20221107/subreddits/text',
 'folder_posts_text_and_meta': 'i18n_topic_model_batch/runs/20221107/posts',
 'folder_comments_text_and_meta': 'i18n_topic_model_batch/runs/20221107/comments',
 'folder_post_and_comment_text_and_meta': 'i18n_topic_model_batch/runs/20221107/post_and_comment_text_combined/text_all'}

In [16]:
# data with embeddings
cfg_cluster_test.config_dict['data_embeddings_to_cluster']

{'config_description': 'v0.6.1 config for embeddings. C1= Post & comment text were combined and created as single embedding (no averaging).',
 'run_uuid': '91ac7ca171024c779c0992f59470c81b',
 'l_ix_sub': ['subreddit_name', 'subreddit_id'],
 'l_ix_post': ['subreddit_name', 'post_id'],
 'df_post_level_agg_b_post_and_comments': None,
 'df_post_level_agg_c_post_comments_sub_desc': None,
 'df_post_level_agg_c1_post_comments_sub_desc': 'df_posts_agg_c1',
 'df_sub_level_agg_a_post_only': None,
 'df_sub_level_agg_b_post_and_comments': None,
 'df_sub_level_agg_c_post_comments_and_sub_desc': None,
 'df_sub_level_agg_c1_post_comments_and_sub_desc': 'df_subs_agg_c1'}

In [17]:
# clustering algo
cfg_cluster_test.config_dict['clustering_algo']

{'model_name': 'AgglomerativeClustering',
 'connectivity_graph': {'add_step': False,
  'name': 'kneighbors_graph',
  'kwargs': {'n_neighbors': 30,
   'mode': 'connectivity',
   'metric': 'cosine',
   'include_self': False,
   'n_jobs': 10}},
 'model_kwargs': {'n_clusters': 65,
  'affinity': 'euclidean',
  'linkage': 'ward',
  'compute_distances': True,
  'compute_full_tree': True}}

# Run as python fnx

In [18]:
mlflow.end_run('KILLED')

In [19]:
%%time

# use ** to over-ride config defaults
cls_test = ClusterEmbeddings(
    **{
        **cfg_cluster_test.config_dict,
        **{
            'n_sample_embedding_rows': 9000,
            'mlflow_experiment_name': test_experiment,
        }
    }
)

df_emb_ = cls_test._load_embeddings()
# cls_test.run_clustering()

15:39:33 | INFO | "Local folder to download artifact(s):
  /home/jupyter/subreddit_clustering_i18n/data/local_cache/i18n-subreddit-clustering/mlflow/mlruns/40/91ac7ca171024c779c0992f59470c81b/artifacts/df_subs_agg_c1"
100%|########################################| 12/12 [00:00<00:00, 36498.66it/s]
15:39:33 | INFO | "  Parquet files found:     4"
15:39:33 | INFO | "  Parquet files to use:     4"
15:39:55 | INFO | "  SAMPLING n_rows: 9000"
15:39:56 | INFO | "    9,000 |   515 <- df_embeddings SHAPE"


CPU times: user 10 s, sys: 6.38 s, total: 16.4 s
Wall time: 29 s


In [22]:
df_sub_ = cls_test._load_metadata_for_filtering(df_emb_)

15:42:33 | INFO | "Filter column already in df_embeddings, no need to load metadata. posts_for_embeddings_count"


In [23]:
df_sub_

""


In [20]:
df_emb_.iloc[:5:, :10]

,subreddit_id,subreddit_name,posts_for_embeddings_count,embeddings_0,embeddings_1,embeddings_2,embeddings_3,embeddings_4,embeddings_5,embeddings_6
673557,t5_7av381,oni_doji,2,0.030877,0.035704,0.026931,0.027594,-0.009047,0.037756,-0.019804
237772,t5_5qs1wa,lulubbc,1,-0.009214,-0.034508,0.026211,0.037649,0.014410,0.029529,-0.014218
26675,t5_2iyagl,drugcombinations,3,-0.034336,0.012822,-0.022650,-0.009457,-0.033129,0.068856,-0.011272
723643,t5_7befs6,bukyzone,0,-0.031158,-0.027359,0.009492,-0.029970,0.080421,0.039674,0.002206
630018,t5_78mdku,pupliccommunity1211,2,0.028190,-0.019427,0.036023,0.020662,0.014316,0.067800,-0.012519


# Run commmand line fxn

The clustering fxn is in `subclu.models.conlustering.py`

Notes:
- We need to use the `-m` flag to run as a submodule (and allow relative imports)
- When using the `-m` flag, REMOVE the `.py` ending of the file!
- In the command line for hydra we can override w/o having to use the `+` sign.
    - https://hydra.cc/docs/tutorials/basic/your_first_app/config_file

## Check & set paths

In [43]:
test_experiment

'v0.5.0_mUSE_clustering_test'

In [44]:
os.getcwd()

'/home/jupyter/subreddit_clustering_i18n/notebooks/v0.5.0'

In [45]:
path_djb_repo = '/home/david.bermejo/repos/subreddit_clustering_i18n/' 
path_djb_models = '/home/david.bermejo/repos/subreddit_clustering_i18n/subclu/models' 
file_clustering_py = 'subclu.models.clustering'

config_name = 'clustering_v0.5.0_subreddit_base'

## Run clustering from CLI

```bash
!cd $path_djb_repo && python -m $file_clustering_py \
    mlflow_experiment_name=$test_experiment \
    n_sample_embedding_rows=4000 \
    filter_embeddings.filter_subreddits.minimum_column_value=9
```

In [46]:
file_clustering_py

'subclu.models.clustering'

In [47]:
config_name

'clustering_v0.5.0_subreddit_base'

In [49]:
# run on sample data, test experiment

!cd $path_djb_repo && python -m $file_clustering_py \
    --config-name $config_name \
    mlflow_experiment_name=$test_experiment \
    n_sample_embedding_rows=9000 \
    pipeline_config.reduce.add_step=true \
    filter_embeddings.filter_subreddits.minimum_column_value=8

CFG keys: dict_keys(['data_text_and_metadata', 'data_embeddings_to_cluster', 'clustering_algo', 'embeddings_to_cluster', 'n_sample_embedding_rows', 'n_max_clusters_to_check_for_optimal_k', 'filter_embeddings', 'mlflow_tracking_uri', 'mlflow_experiment_name', 'pipeline_config'])
`2022-07-06 17:38:59,768` | `INFO` | `Define cluster class...`
`2022-07-06 17:39:00,714` | `INFO` | `== Start run_aggregation() method ==`
`2022-07-06 17:39:00,714` | `INFO` | `MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-100-2021-04-28-djb-eda-german-subs/mlruns.db`
`2022-07-06 17:39:00,861` | `INFO` | `=== START CLUSTERING - Process ID 49697`
`2022-07-06 17:39:00,958` | `INFO` | `host_name: djb-100-2021-04-28-djb-eda-german-subs`
`2022-07-06 17:39:00,958` | `INFO` | `cpu_count: 96`
`2022-07-06 17:39:01,027` | `INFO` | `RAM stats:
{'memory_used_percent': '1.91%', 'memory_total': '1,444,961', 'memory_used': '27,629', 'memory_free': '1,414,564'}`
`2022-07-06 17:39:01,137`

In [65]:
# Since we're including 80k subreddits for modeling, we might need to increase the # of subclusters
n_subreddits_ = 81000

for n_ in [100, 400, 1000, 2000, 4000, 8000]:
    print(f"n={n_:6,.0f} -> {n_subreddits_ / n_:6,.1f} subs per cluster")

n=   100 ->  810.0 subs per cluster
n=   400 ->  202.5 subs per cluster
n= 1,000 ->   81.0 subs per cluster
n= 2,000 ->   40.5 subs per cluster
n= 4,000 ->   20.2 subs per cluster
n= 8,000 ->   10.1 subs per cluster


## Test multirun

Jobs can run in parallel with `--multirun` flag!

In [66]:
print(fr"""
cd {path_djb_repo} && python -m {file_clustering_py} --multirun \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 10, 2)" \
    mlflow_experiment_name={test_experiment} \
    n_sample_embedding_rows=4200
""")


cd /home/david.bermejo/repos/subreddit_clustering_i18n/ && python -m subclu.models.clustering --multirun \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 10, 2)" \
    mlflow_experiment_name=v0.5.0_mUSE_clustering_test \
    n_sample_embedding_rows=4200



### Example: 7 jobs, 5 at the same time

With this command:

```bash
!cd $path_djb_repo && python -m $file_clustering_py --multirun \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 9, 1)" \
    mlflow_experiment_name=$test_experiment \
    n_sample_embedding_rows=4200
```

We can adjust `n_jobs` (how many in parallel) in the root config:
```yaml
# config: config/clustering_v0.4.1_subreddit_base.yaml

# Change n_jobs(parallel jobs) & logging for hydra itself
hydra:
  launcher:
    # override the number of jobs for joblib
    n_jobs: 5
```

With n_job=5, we expect:
- 7 jobs total
    - because we'll change the `filter_embeddings.filter_subreddits.minimum_column_value` parameter to:
        - Expanded range = [2, 3, 4, 5, 6, 7, 8]
- 2 batches of jobs
    - Each batch = 5 jobs in parallel
        - batch 1: job # IDs: 0, 1, 2, 3, 4
        - batch 2: job # IDs: 5, 6

In [67]:
# # run on sample data, multi-run

# !cd $path_djb_repo && python -m $file_clustering_py --multirun \
#     "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 9, 1)" \
#     "hydra.launcher.n_jobs=3" \
#     mlflow_experiment_name=$test_experiment \
#     n_sample_embedding_rows=4200

### Results after making the `cluster_embeddings` function return nothing:

Success!!! 7 jobs ran in 2 batches without any errors!

## Run on full data (one job)

In [40]:
# # run on full data, still a test

# !cd $path_djb_repo && python -m $file_clustering_py mlflow_experiment_name=$test_experiment

In [56]:
# # # run on full data, no longer a test

# !cd $path_djb_repo && python -m $file_clustering_py

## Run on full data (multijob)

In [69]:
print(path_djb_repo)
print(config_name)

/home/david.bermejo/repos/subreddit_clustering_i18n/
clustering_v0.5.0_subreddit_base


In [70]:
# ward-related jobs
!cd $path_djb_repo && python -m $file_clustering_py --multirun \
    --config-name $config_name \
    "clustering_algo.model_kwargs.linkage='ward'" \
    "clustering_algo.model_kwargs.affinity='euclidean'" \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 5, 1)" \
    "pipeline_config.normalize.add_step=choice(false, true)" \
    "pipeline_config.reduce.add_step=choice(false, true)"

[2022-07-06 17:59:53,387][HYDRA] Joblib.Parallel(n_jobs=11,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 12 jobs
[2022-07-06 17:59:53,387][HYDRA] Launching jobs, sweep output dir : /home/jupyter/subreddit_clustering_i18n/hydra_runs/multirun/2022-07-06/17-59-51
[2022-07-06 17:59:53,387][HYDRA] 	#0 : clustering_algo.model_kwargs.linkage='ward' clustering_algo.model_kwargs.affinity='euclidean' filter_embeddings.filter_subreddits.minimum_column_value=2 pipeline_config.normalize.add_step=False pipeline_config.reduce.add_step=False
[2022-07-06 17:59:53,387][HYDRA] 	#1 : clustering_algo.model_kwargs.linkage='ward' clustering_algo.model_kwargs.affinity='euclidean' filter_embeddings.filter_subreddits.minimum_column_value=2 pipeline_config.normalize.add_step=False pipeline_config.reduce.add_step=True
[2022-07-06 17:59:53,387][HYDRA] 	#2 : clustering_algo.model_kwargs.linkage='ward

In [71]:
# non-ward linkages
#  We set normalize.add_ste=false because that's only used for Ward,
#  where normalize+euclidean = cosine distance

!cd $path_djb_repo && python -m $file_clustering_py --multirun \
    --config-name $config_name \
    "clustering_algo.model_kwargs.linkage=choice('complete', 'average')" \
    "clustering_algo.model_kwargs.affinity=choice('cosine', 'euclidean')" \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 5, 1)" \
    "pipeline_config.reduce.add_step=choice(false, true)" \
    "pipeline_config.normalize.add_step=false"

[2022-07-06 18:44:24,468][HYDRA] Joblib.Parallel(n_jobs=11,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 24 jobs
[2022-07-06 18:44:24,468][HYDRA] Launching jobs, sweep output dir : /home/jupyter/subreddit_clustering_i18n/hydra_runs/multirun/2022-07-06/18-44-21
[2022-07-06 18:44:24,468][HYDRA] 	#0 : clustering_algo.model_kwargs.linkage=complete clustering_algo.model_kwargs.affinity=cosine filter_embeddings.filter_subreddits.minimum_column_value=2 pipeline_config.reduce.add_step=False pipeline_config.normalize.add_step=False
[2022-07-06 18:44:24,468][HYDRA] 	#1 : clustering_algo.model_kwargs.linkage=complete clustering_algo.model_kwargs.affinity=cosine filter_embeddings.filter_subreddits.minimum_column_value=2 pipeline_config.reduce.add_step=True pipeline_config.normalize.add_step=False
[2022-07-06 18:44:24,468][HYDRA] 	#2 : clustering_algo.model_kwargs.linkage=complete cl